In [ ]:
import numpy as np
import gurobipy as gp
import pandas as pd
import seaborn as sns   # Why sns?  It's a reference to The West Wing
import matplotlib.pyplot as plt  # seaborn is based on matplotlib
sns.set(color_codes=True) # adds a nice background to the graphs
%matplotlib inline

In [ ]:
nnodes = 25
xy = np.random.normal(size=(nnodes,2))
dist = np.zeros((nnodes,nnodes))
for i in range(nnodes):
    for j in range(nnodes):
        dist[i,j] = np.sqrt((xy[i,0]-xy[j,0])**2 + (xy[i,1]-xy[j,1])**2)
        if i == j:
            dist[i,j] = 10000
            
df = pd.DataFrame({'x':xy[:,0],'y':xy[:,1]})
sns.scatterplot(data=df,x='x',y='y');


In [ ]:
obj = np.append(np.reshape(dist,nnodes**2),np.array([0]*nnodes))

In [ ]:
N = nnodes
A = np.zeros((2*N + (N-1)**2 - (N-1),N**2 + N))
b = np.zeros(2*N + (N-1)**2 - (N-1))
direction = np.array(['']*(2*N + (N-1)**2 - (N-1)))

In [ ]:
ind_vec = np.array(range(N))
# leave each city once
row = 0
for j in range(N):
    A[row,j*N + ind_vec] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [ ]:
# enter each city once
for i in range(N):
    A[row,ind_vec*N + i] = 1
    b[row] = 1
    direction[row] = '='
    row+=1

In [ ]:
for i in range(1,N):
    for j in range(1,N):
        if i != j:
            # ui - uj + Nxij <= (N-1)
            A[row,[N**2+i,N**2+j,j*N+i]] = [1,-1,N]
            b[row] = N-1
            direction[row] = '<'
            row+=1

In [ ]:
tspMod = gp.Model()
tspMod_x = tspMod.addMVar(N**2 + N,vtype=['B']*(N**2)+['C']*N) 
tspMod_con = tspMod.addMConstrs(A, tspMod_x, direction, b)
tspMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

tspMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
tspMod.Params.TimeLimit = 120
tspMod.optimize()

In [ ]:
tspMod.ObjVal

In [ ]:
sns.scatterplot(data=df,x='x',y='y');
for i in range(N):
    for j in range(N):
        if tspMod_x.x[i*N+j] > 0.9:
            plt.plot([xy[i,0],xy[j,0]],[xy[i,1],xy[j,1]],'r')

# simulated annealing

In [ ]:
def transport(path):
    nx = len(path)
    startstop = np.random.choice(nx,2,replace=False)
    if startstop[0] > startstop[1]:
        remove_path = np.append(path[startstop[0]:],path[:(startstop[1]+1)])
        closed_path = path[(startstop[1]+1):startstop[0]]
    else:
        remove_path = path[startstop[0]:(startstop[1]+1)]
        closed_path = np.append(path[0:startstop[0]],path[(startstop[1]+1):])
    nc = len(closed_path)
    if nc > 0:
        paste = np.random.choice(nc,1)[0]
        newpath = np.append(closed_path[0:(paste+1)],remove_path)
        newpath = np.append(newpath,closed_path[(paste+1):])
    else:
        newpath = path
    return newpath

In [ ]:
def reverse(path):
    nx = len(path)
    startstop = np.random.choice(nx,2,replace=False)
    if startstop[0] > startstop[1]:
        remove_path = np.append(path[startstop[0]:],path[:(startstop[1]+1)])
        remove_path = remove_path[::-1]
        closed_path = path[(startstop[1]+1):startstop[0]]
        new_path = np.append(closed_path,remove_path)
    else:
        remove_path = path[startstop[0]:(startstop[1]+1)]
        remove_path = remove_path[::-1]
        new_path = np.append(path[0:startstop[0]],remove_path)
        new_path = np.append(new_path,path[(startstop[1]+1):])
    return new_path   

In [ ]:
def measure_path(path,dist):
    nx = len(path)
    total_dist = 0
    for i in range(nx-1):
        total_dist += dist[path[i],path[i+1]]
    total_dist += dist[path[nx-1],path[0]]
    return total_dist

In [ ]:
eps = 1
nloop = 100000
delta = eps/nloop
path = np.random.choice(nnodes,nnodes,False)
total_dist = measure_path(path,dist)
for j in range(nloop):
    if np.random.random() < 0.5:
        newpath = transport(path)
    else:
        newpath = reverse(path)
    newdist = measure_path(newpath,dist)
    if newdist < total_dist:
        path = newpath
        total_dist = newdist
    else:
        if np.random.random() < eps/np.exp(newdist-total_dist):
            path = newpath
            total_dist = newdist
    eps -= delta
    if (j % 500) == 0:
        print(j,total_dist)

In [ ]:
sns.scatterplot(data=df,x='x',y='y');
for i in range(N-1):
    plt.plot([xy[path[i],0],xy[path[i+1],0]],[xy[path[i],1],xy[path[i+1],1]],'r')
plt.plot([xy[path[N-1],0],xy[path[0],0]],[xy[path[N-1],1],xy[path[0],1]],'r');

In [ ]:
sns.scatterplot(data=df,x='x',y='y');
for i in range(N):
    for j in range(N):
        if tspMod_x.x[i*N+j] > 0.9:
            plt.plot([xy[i,0],xy[j,0]],[xy[i,1],xy[j,1]],'r')